In [9]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import ticker
import seaborn as sns
import os
import numpy as np
from sklearn import datasets


In [10]:
%store -r df_combined

In [11]:
df_combined

,Relative Compactness,Surface Area,Wall Area,Roof Area,Overall Height,Glazing Area,Heating Load,Orientation Dummy,Glazing Area Distribution Dummy
0,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0
1,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0
2,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0
3,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0
4,0.98,514.5,294.0,110.25,7.0,0.0,15.55,0,0
...,...,...,...,...,...,...,...,...,...
15325,0.62,808.5,367.5,220.50,3.5,0.4,16.64,1,0
15326,0.62,808.5,367.5,220.50,3.5,0.4,16.64,1,0
15327,0.62,808.5,367.5,220.50,3.5,0.4,16.64,1,0
15328,0.62,808.5,367.5,220.50,3.5,0.4,16.64,1,0
